In [1]:
%%loadFromPOM
<!-- support http patch -->
<dependency>
    <groupId>org.apache.httpcomponents</groupId>
    <artifactId>httpclient</artifactId>
    <version>4.5.3</version>
</dependency>

<!-- https://mvnrepository.com/artifact/org.springframework/spring-web -->
<dependency>
    <groupId>org.springframework</groupId>
    <artifactId>spring-web</artifactId>
    <version>5.3.19</version>
</dependency>


----- printer init -----



In [1]:
String a = "a"


----- printer init -----



In [3]:
var a = "aaa"

In [4]:
a = 1

CompilationException: 

In [18]:
null instanceof Boolean

false

### 1. url build

In [2]:
import org.springframework.web.util.UriComponentsBuilder;

In [10]:
UriComponentsBuilder builder = UriComponentsBuilder.fromHttpUrl("http://host:8080/a").path("1").path("/2");
builder.queryParam("a", "a").queryParam("b", "b b");
print(builder.build().toUriString())

printer| var-7: http://host:8080/a1/2?a=a&b=b b


In [6]:
UriComponentsBuilder builder = UriComponentsBuilder.fromHttpUrl("host:8080/a/").path("1").path("/2");
builder.queryParam("a", "a").queryParam("b", "b b");
print(builder.toUriString())

CompilationException: 

### 2. send

In [ ]:
private static final ScheduledExecutorService scheduledExecutorService = Executors.newSingleThreadScheduledExecutor();
    private static final int CONNECTION_MAX_TOTAL = 100;

In [ ]:
//忽略HTTPS证书
TrustStrategy trustStrategy = (x509Certificates, authType) -> true;
SSLContext sslContext = null;
try{
    sslContext = SSLContexts.custom().loadTrustMaterial(null, trustStrategy).build();
} catch (Exception e){
    log.error(e.getMessage(), e);
}
Objects.requireNonNull(sslContext);
Registry<ConnectionSocketFactory> registry = RegistryBuilder.<ConnectionSocketFactory>create()
        .register("http", PlainConnectionSocketFactory.getSocketFactory())
        .register("https", new SSLConnectionSocketFactory(sslContext, new NoopHostnameVerifier()))
        .build();
//使用连接池
PoolingHttpClientConnectionManager connectionManager = new PoolingHttpClientConnectionManager(registry);
connectionManager.setMaxTotal(CONNECTION_MAX_TOTAL);//总的最大连接数
connectionManager.setDefaultMaxPerRoute(CONNECTION_MAX_TOTAL);//每host

HttpClient httpClient = HttpClientBuilder.create()
        .setSSLContext(sslContext).setSSLHostnameVerifier(new NoopHostnameVerifier())
        .setConnectionManager(connectionManager).evictExpiredConnections()
        .build();

//SimpleClientHttpRequestFactory factory = new SimpleClientHttpRequestFactory();
HttpComponentsClientHttpRequestFactory factory = new HttpComponentsClientHttpRequestFactory(httpClient);
//建立连接后读取数据超时时间
factory.setReadTimeout(60000);//ms
//建立连接超时时间
factory.setConnectTimeout(5000);//ms

restTemplate = new RestTemplate(factory);
MappingJackson2HttpMessageConverter converter = new MappingJackson2HttpMessageConverter();
converter.getObjectMapper().setPropertyNamingStrategy(PropertyNamingStrategy.SNAKE_CASE);
converter.getObjectMapper().setSerializationInclusion(JsonInclude.Include.NON_NULL);
converter.getObjectMapper().configure(DeserializationFeature.FAIL_ON_UNKNOWN_PROPERTIES, false);
restTemplate.getMessageConverters().add(0, converter);

List<ClientHttpRequestInterceptor> interceptors = new ArrayList<>();
interceptors.add((request, body, execution) -> {
    log.debug("request stack url = {}", request.getURI());
    if (body.length > 0) {
        log.debug("request stack body={}", new String(body, "utf-8"));
    }
    request.getHeaders().setContentType(MediaType.APPLICATION_JSON);
    if (isRequestNeedAuth(request)) {
        addAuthTokenHeader(request);
    }
    ClientHttpResponse response = execution.execute(request, body);
    return response;
});
restTemplate.setInterceptors(interceptors);
//自动刷新token,每2分钟执行1次，token在过期时间前10分钟刷新
scheduledExecutorService.scheduleWithFixedDelay(new Runnable() {
    @Override
    public void run() {
        try {
            refreshTokenIfNeed();
        } catch (Exception e) {
            log.error(e.getMessage(), e);
        }
    }
}, 2, 120, TimeUnit.SECONDS);

In [ ]:
ParameterizedTypeReference<List<JobInfo>> reference = new ParameterizedTypeReference<List<JobInfo>>() {
};
ResponseEntity<List<JobInfo>> forEntity = restTemplate.exchange(url, HttpMethod.GET, null, reference);